In [37]:
import math
import random
import torch

In [38]:
class Value:
    def __init__(self, data, _children=(), op='', label=''):
        self.data = data
        self.grad = 0.0
        self.backward = lambda :None
        self.prev = set(_children)
        self.op = op
        self.label = label

    def __repr__(self):
        return f"Value(Data={self.data})"

    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data + other.data, (self, other),'+')
        def backward():
            self.grad += 1.0 * out.grad
            other.grad += 1.0 * out.grad
        out.backward = backward
        return out

    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, (self, other),'*')
        def backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        out.backward = backward
        return out

    def __pow__(self, other):
        assert isinstance(other, (int, float)) # only support int, float
        out = Value(self.data**other, (self, ), f"**{other}")
        def backward():
            self.grad += (other * self.data ** (other - 1)) * out.grad
        out.backward = backward
        return out

    def __radd__(self, other):
        return self * other

    def __rmul__(self, other): # other * self
        return self * other

    def __rsub__(self, other):
        return self * other

    def __truediv__(self, other): # self / other
        return self * other**-1

    def __neg__(self):
        return self * (-1)

    def __sub__(self, other):
        return self + (-other)

    def tanh(self):
        num = self.data
        t = (math.exp(2*num) - 1)/(math.exp(2*num) + 1)
        out = Value(t, (self, ), 'tanh')
        def backward():
            self.grad += (1 - t**2) * out.grad
        out.backward = backward
        return out

    def exp(self):
        x = self.data
        out = Value(math.exp(x), (self, ), 'exp')
        def backward():
            self.grad = out.data * out.grad
        out.backward = backward
        return out

    def back_ward(self):
        topo = []
        visited  = set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v.prev:
                    build_topo(child)
                topo.append(v)
        build_topo(self)

        self.grad = 1.0
        for node in reversed(topo):
            node.backward()

In [39]:
# Backpropagation in PyTorch
x1 = torch.Tensor([2.0]).double();   x1.requires_grad = True
x2 = torch.Tensor([0.0]).double();   x2.requires_grad = True
w1 = torch.Tensor([-3.0]).double();  w1.requires_grad  =True
w2 = torch.Tensor([1.0]).double();   w2.requires_grad = True
b = torch.Tensor([6.8813735870195432]).double();  b.requires_grad = True

# forward pass
x1w1 = x1*w1
x2w2 = x2*w2
n = x1w1 + x2w2 + b
o = torch.tanh(n)

# backpropagation
o.retain_grad()
o.backward()

print("x1",x1.grad.item())
print("x2",x2.grad.item())
print("w1",w1.grad.item())
print("w2",w2.grad.item())

x1 -1.5000003851533106
x2 0.5000001283844369
w1 1.0000002567688737
w2 0.0


In [40]:
class Neuron:
    def __init__(self, num_of_inputs):
        self.weight = [Value(random.uniform(-1, 1)) for _ in range(num_of_inputs)]
        self.bias = Value(random.uniform(-1, 1))
    def __call__(self, x):
        # w * x + b
        act = sum((wi * xi for wi, xi in zip(self.weight, x)), self.bias)
        out = act.tanh()
        return out
    def parameters(self):
        return self.weight + [self.bias]
put = [2.0, 3.0]
nu = Neuron(2)
nu(put)

Value(Data=0.9909978521458636)

In [41]:
class Layer:
    def __init__(self, number_of_inputs, number_of_outputs):
        self.neurons = [Neuron(number_of_inputs) for _ in range(number_of_outputs)]
    def __call__(self, takes):
        outs = [neuron(takes) for neuron in self.neurons]
        return outs[0] if len(outs) == 1 else outs
    def parameters(self):
        return [p for neuron in self.neurons for p in neuron.parameters()]
        # params = []
        # for neuron in self.neurons:
        #     ps = neuron.parameters()
        #     params.extend(ps)
        # return params
put_1 = [2.0, 3.0]
l = Layer(2, 3)
l(put_1)

[Value(Data=0.5892638903679766),
 Value(Data=-0.5759746137542763),
 Value(Data=0.975638677717391)]

In [42]:
class MLP:
    def __init__(self, number_of_inputs, number_of_outputs):
        size = [number_of_inputs] + number_of_outputs
        print(number_of_outputs)
        print(size)
        self.layers = [Layer(size[i], size[i + 1]) for i in range(len(number_of_outputs))]
        for num in range(len(number_of_outputs)):
            print(size[num])
            print(size[num + 1])
    def __call__(self, puts):
        for layer in self.layers:
            puts = layer(puts)
        return puts
    def parameters(self):
        return [por for layer in self.layers for por in layer.parameters()]

In [43]:
inputs = [2.0, 3.0, -1.0]
n = MLP(3, [4, 4, 1])
n(inputs)

[4, 4, 1]
[3, 4, 4, 1]
3
4
4
4
4
1


Value(Data=0.5422558376560979)

In [44]:
n.parameters()

[Value(Data=-0.5250479502612884),
 Value(Data=0.3557309216974993),
 Value(Data=0.9069944272455412),
 Value(Data=0.35155928251085333),
 Value(Data=-0.22353837447483538),
 Value(Data=-0.9957150706787461),
 Value(Data=-0.3018264827772088),
 Value(Data=-0.040154266594470744),
 Value(Data=0.029674580254776872),
 Value(Data=-0.6876394361270022),
 Value(Data=0.03937719147415275),
 Value(Data=0.09650773059806994),
 Value(Data=0.13644806463138592),
 Value(Data=0.5854276946577672),
 Value(Data=0.34656799657499127),
 Value(Data=0.05075900609717454),
 Value(Data=-0.3604303641670732),
 Value(Data=-0.912347829342695),
 Value(Data=0.3679324966846991),
 Value(Data=0.13055565948252257),
 Value(Data=-0.9579944983633042),
 Value(Data=0.9729323424497427),
 Value(Data=0.12443163637110488),
 Value(Data=-0.7672279399702537),
 Value(Data=0.9124308380531709),
 Value(Data=-0.6134724091376198),
 Value(Data=-0.6814899660618976),
 Value(Data=0.9475333389175131),
 Value(Data=0.06530700253213984),
 Value(Data=0.7841

In [45]:
xs = [
    [2.0, 3.0, -1.0],
    [3.0, -1.0, 0.5],
    [0.5, 1.0, 1.0],
    [1.0, 1.0, -1.0]
]
ys = [1.0, -1.0, -1.0, 1.0]

In [46]:
for k in range(50):
    # forward pass
    ypred = [n(x) for x in xs]
    loss = sum((yout - ygt)**2 for ygt, yout in zip(ys, ypred))

    # backward pass
    for p in n.parameters():
        p.grad = 0.0
    loss.back_ward()

    # update
    for p in n.parameters():
        p.data += -0.05 * p.grad
    print(k, loss.data)

0 2.9570041334120947
1 1.3046821952094012
2 0.5353498352157795
3 0.33032242945342494
4 0.23044456644834757
5 0.1722542347430465
6 0.13516671686780085
7 0.10994700882683087
8 0.09191852707827776
9 0.07851180737048949
10 0.06822026816445917
11 0.06011205657049909
12 0.05358457050985306
13 0.04823324218846568
14 0.04377765607878963
15 0.040018050299798526
16 0.036808697868823584
17 0.03404105973944515
18 0.03163280259073868
19 0.029520450337307005
20 0.027654351569446
21 0.025995160741450772
22 0.0245113314333592
23 0.023177300227449862
24 0.021972150640750143
25 0.020878616414298976
26 0.019882328413754966
27 0.018971238891758054
28 0.018135176564293254
29 0.01736549933120793
30 0.016654820693696767
31 0.015996792369543433
32 0.015385930174127661
33 0.014817473509751308
34 0.014287271180304945
35 0.01379168798841826
36 0.013327527860192711
37 0.012891970204818245
38 0.012482516941546003
39 0.012096948177542033
40 0.011733284942173308
41 0.0113897577088728
42 0.01106477968868734
43 0.0107

In [47]:
ypred

[Value(Data=0.6852318366887756),
 Value(Data=-0.9509266975304742),
 Value(Data=-0.937999478440863),
 Value(Data=0.9456585935445728)]